# Imports

**NOTE: Make sure to use the get_properties_environment file to set your conda environment.**

In [1]:
import os,re,sys,pickle,datetime,time,random,itertools,glob
import warnings
warnings.filterwarnings("ignore")
import openpyxl
import pandas as pd
from rdkit import Chem
import get_properties_functions as gp

#acid-specific function for octant-derived parameters (and required imports)
from morfeus import BuriedVolume

def get_vbur_acid_octant_params(dataframe, a1, z_atom, xz_plane_atom, radius): #uses morfeus to calculate vbur at a single radius, divided into hemispheres defined by two other atoms
    atom = str(a1)
    vbur_octant_dataframe = pd.DataFrame(columns=[])
    
    for index, row in dataframe.iterrows():
        try:
            log_file = row['log_name']
            atom1 = row[str(a1)] #gets numerical value (e.g., 16) for a1 (e.g., C1)
            atom2 = row[str(z_atom)]
            atom3 = row[str(xz_plane_atom)]
            
            streams, error = gp.get_outstreams(log_file) #need to add file path if you're running from a different directory than file
            if error != "":
                print(error)
                row_i = {'%Vbur_'+str(atom)+"_min_hemisphere_"+str(radius)+"Å": "no data", 
                        '%Vbur_'+str(atom)+"_max_hemisphere_"+str(radius)+"Å": "no data"}
                vbur_octant_dataframe = vbur_octant_dataframe.append(row_i, ignore_index=True)
                continue
            
            log_coordinates = gp.get_geom(streams)
            elements = np.array([log_coordinates[i][0] for i in range(len(log_coordinates))])
            coordinates = np.array([np.array(log_coordinates[i][1:]) for i in range(len(log_coordinates))])
            vbur = BuriedVolume(elements, coordinates, int(atom1), include_hs=True, radius=radius, z_axis_atoms=[int(atom2)], xz_plane_atoms=[int(atom3)]) #calls morfeus
            vbur.octant_analysis()
            #vbur.plot_steric_map()
            percent_vbur_quads = vbur.quadrants['percent_buried_volume']
            hem1 = (percent_vbur_quads[1] + percent_vbur_quads[2])/2 #these are the hemispheres bisected by the plane of the input atoms
            hem2 = (percent_vbur_quads[3] + percent_vbur_quads[4])/2
            hem_list = [hem1, hem2]
            
            percent_vbur_octs = vbur.octants['percent_buried_volume']
            quad1 = (percent_vbur_octs[0] + percent_vbur_octs[1])/2 #these are the quadrants below C1 (away from the carbonyl), bisected by the plane of C1, C4, O2
            quad2 = (percent_vbur_octs[2] + percent_vbur_octs[3])/2 #mimics the Burgi-Dunitz angle
            quad_list = [quad1, quad2]
            
            row_i = {'%Vbur_'+str(atom)+"_min_hemisphere_"+str(radius)+"Å": min(hem_list), 
                    '%Vbur_'+str(atom)+"_max_hemisphere_"+str(radius)+"Å": max(hem_list)}
            vbur_octant_dataframe = vbur_octant_dataframe.append(row_i, ignore_index=True)
        except:
            print('****Unable to acquire hemisphere Vbur parameters for:', row['log_name'], ".log")
            row_i = {'%Vbur_'+str(atom)+"_min_hemisphere_"+str(radius)+"Å": "no data", 
                    '%Vbur_'+str(atom)+"_max_hemisphere_"+str(radius)+"Å": "no data" }
            vbur_octant_dataframe = vbur_octant_dataframe.append(row_i, ignore_index=True)
    return(pd.concat([dataframe, vbur_octant_dataframe], axis = 1))

D3 import failed


# Atom Inputs Dataframe

## Generate dataframe with atom numbers

### Use command line or bash script to prepare files

To create files: navigate to folder that contains all the log files you wish to analyze.

    obabel *.log -osdf -m  
    ls *.log > log_ids.txt
    cat *.sdf >> molecules.sdf

### Define SMARTS substructure


Recommended to draw the common substructure (with general atoms) in Chemdraw and copy as SMILES (this will generate a SMARTS string)

More information about SMARTS and available characters here: https://www.daylight.com/dayhtml/doc/theory/theory.smarts.html


In [2]:
#this is the common smarts substructure for the molecules you will analyze
#you have to explicitly draw hydrogens into the SMARTS structure if you want to collect properties for hydrogen atoms
substructure = Chem.MolFromSmarts('[*]C(O[H])=O')

### Generate preliminary dataframe

In [3]:
#generate a list of molecules using RDkit
all_compounds = Chem.SDMolSupplier('molecules.sdf', removeHs=False) 
#molecules.sdf is generated with the instructions above
#it is a single sdf that contains the structures/atom numbers etc. for every molecule you will analyze

#uses RDKit to search for the substructure in each compound you will analyze
atoms = []
for molecule in all_compounds:
    if molecule is not None:
        submatch = molecule.GetSubstructMatches(substructure) #find substructure
        matchlist = list([item for sublist in submatch for item in sublist]) #list of zero-indexed atom numbers
        match_idx = [x+1 for x in matchlist] #this line changes from 0-indexed to 1-indexed (for Gaussian)
        atoms.append(match_idx) #append 1-indexed list to atoms (a list of lists)
        
#this loop extracts log names from log_ids and splits them to the desired format
filenames = open("log_ids.txt", "r") #generate this with instruction above
#it is a text file that contains the file name for every molecule you will analyze
list_of_filenames = [(line.strip()).split() for line in filenames] #list of the file names (each of which includes all conformers)
list_of_files = []
for filename in list_of_filenames:
    file = filename[0].split(".")
    list_of_files.append(file[0])
filenames.close()

#put the atom numbers for the substructure for each log file into a dataframe
prelim_df = pd.DataFrame(atoms) 
index=list_of_files
prelim_df.insert(0,column='log_name',value=list_of_files)
display(prelim_df)

,log_name,0,1,2,3,4
0,Ac1_clust-1,3,2,1,13,12
1,Ac1_clust-10,3,2,1,13,12
2,Ac1_clust-11,3,2,1,13,12
3,Ac1_clust-12,3,2,1,13,12
4,Ac1_clust-13,3,2,1,13,12
5,Ac1_clust-14,3,2,1,13,12
6,Ac1_clust-15,3,2,1,13,12
7,Ac1_clust-16,3,2,1,13,12
8,Ac1_clust-17,3,2,1,13,12
9,Ac1_clust-18,3,2,1,13,12


### Define column headers using GaussView

Using the preliminary dataframe displayed above, open one of your files and check the atom numbers. 

Assign labels to each column using the cell below. You will call these column headers when you select your properties. 

**User input required:**

In [4]:
atom_labels = {'log_name': 'log_name',
                0: 'C4',
                1: 'C1', 
                2: 'O3',
                3: 'H5',
                4: 'O2'}

### Generate labeled dataframe

**NOTE: it is very important you assign these correctly otherwise the properties you collect will be for the wrong atoms and not produce meaningful correlations.** 

We recommend checking the numbering/headers for at least two different compounds. 

Numbering for different conformers of the same compounds will likely be the same (but may not be for some symmetrical groups).

In [5]:
#rename columns using the user input above
atom_map_df = prelim_df.rename(columns=atom_labels)
display(atom_map_df.head())

#you can use this to clean up the table if you have more atoms in your substructure than you want to collect descriptors for
#atom_map_df = atom_map_df.drop(columns= ['C4', 'C1']) 
#display(atom_map_df.head())

df = atom_map_df #df is what properties will be appended to, this creates a copy so that you have the original preserved 

,log_name,C4,C1,O3,H5,O2
0,Ac1_clust-1,3,2,1,13,12
1,Ac1_clust-10,3,2,1,13,12
2,Ac1_clust-11,3,2,1,13,12
3,Ac1_clust-12,3,2,1,13,12
4,Ac1_clust-13,3,2,1,13,12


### Save atom map to Excel (if desired)

In [6]:
writer = pd.ExcelWriter('Ac_example_atom_map.xlsx')
atom_map_df.to_excel(writer)
writer.save()

## Import a manually-generated atom mapping dataframe

If you need to manually generate the atom mapping dataframe, check out the atom_map_sample.xlsx to make sure you have the desired format. 

In [7]:
atom_map_df = pd.read_excel('Ac_example_atom_map.xlsx','Sheet1',index_col=0,header=0,engine='openpyxl')
display(atom_map_df.head())
df = atom_map_df #df is what properties will be appended to, this creates a copy so that you have the original preserved 

,log_name,C4,C1,O3,H5,O2
0,Ac1_clust-1,3,2,1,13,12
1,Ac1_clust-10,3,2,1,13,12
2,Ac1_clust-11,3,2,1,13,12
3,Ac1_clust-12,3,2,1,13,12
4,Ac1_clust-13,3,2,1,13,12


# Define Properties to Collect

## Collect properties: 

In [8]:
df = atom_map_df

#---------------GoodVibes Engergies---------------
#uses the GoodVibes 2021 Branch (Jupyter Notebook Compatible)
#calculates the quasi harmonic corrected G(T) and single point corrected G(T) as well as other thermodynamic properties
#inputs: dataframe, temperature
df = gp.get_goodvibes_e(df, 298.15)

#---------------Frontier Orbitals-----------------
#E(HOMO), E(LUMO), mu(chemical potential or negative of molecular electronegativity), eta(hardness/softness), omega(electrophilicity index)
df = gp.get_frontierorbs(df)

#---------------Polarizability--------------------
#Exact polarizability
df = gp.get_polarizability(df)

#---------------Dipole----------------------------
#Total dipole moment magnitude in Debye
df = gp.get_dipole(df)

#---------------Volume----------------------------
#Molar volume
#requires the Gaussian keyword = "volume" in the .com file
df = gp.get_volume(df)

#---------------SASA------------------------------
#Uses morfeus to calculat sovlent accessible surface area and the volume under the SASA
df = gp.get_SASA(df)

#---------------NBO-------------------------------
#natural charge from NBO
#requires the Gaussian keyword = "pop=nbo7" in the .com file
nbo_list = ["C1", "O2", "O3", "C4", "H5"]
df = gp.get_nbo(df, nbo_list) 

#---------------NMR-------------------------------
#isotropic NMR shift
#requires the Gaussian keyword = "nmr=giao" in the .com file
nmr_list = ["C1", "C4", "H5"]
df = gp.get_nmr(df, nmr_list) 

#---------------Distance--------------------------
#distance between 2 atoms
dist_list_of_lists = [["O3", "H5"]]
df = gp.get_distance(df, dist_list_of_lists) 

#---------------Dihedral--------------------------
#dihedral angle between 4 atoms
dihedral_list_of_lists = [["O2", "C1", "O3", "H5"], ["C4", "C1", "O3", "H5"]]
df = gp.get_dihedral(df, dihedral_list_of_lists) 

#---------------IR--------------------------------
#IR frequencies and intensities in a specific range (for specific atoms)
#requires the Gaussian keyword = "freq=noraman" in the .com file
#inputs: dataframe, atom1, atom2, frequency_min, frequency_max, intensity_min, intensity_max, threshold
#if you want to collect multiple IR frequencies, you will need to copy/paste this function for each stretch
#we recommend a threshold of 0.1 or lower (may have to adjust)
df = gp.get_IR(df, "C1", "O2", 1700, 1900, 100, 800, 0.00)

#---------------Vbur Scan-------------------------
#uses morfeus to calculate the buried volume at a series of radii (including hydrogens)
#inputs: dataframe, list of atoms, start_radius, end_radius, and step_size
#if you only want a single radius, put the same value for start_radius and end_radius (keep step_size > 0)
vbur_list = ["C1", "C4"]
df = gp.get_vbur_scan(df, vbur_list, 2, 6, 0.5)

#---------------Octant-Derived Parameters (custom from first cell)-------------------------
df = get_vbur_acid_octant_params(df, "C1", "O2", "C4", 3)
df = get_vbur_acid_octant_params(df, "C1", "O2", "C4", 3.5)
df = get_vbur_acid_octant_params(df, "C1", "O2", "C4", 4)
df = get_vbur_acid_octant_params(df, "C1", "O2", "C4", 4.5)
df = get_vbur_acid_octant_params(df, "C1", "O2", "C4", 5)

#---------------Sterimol morfeus------------------
#uses morfeus to calculate Sterimol L, B1, and B5 values
#NOTE: this is much faster than the corresponding DBSTEP function (recommendation: use as default/if you don't need Sterimol2Vec)
sterimol_list_of_lists = [["C1", "C4"]]
df = gp.get_sterimol_morfeus(df, sterimol_list_of_lists) 

pd.options.display.max_columns = None
display(df)



   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using v

,log_name,C4,C1,O3,H5,O2,E_spc (Hartree),G(T)_spc(Hartree),H_spc(Hartree),T,T*S,T*qh_S,ZPE(Hartree),qh_G(T)_spc(Hartree),HOMO,LUMO,η,μ,ω,polar_aniso(au),polar_iso(au),dipole(Debye),volume(Bohr_radius³/mol),SASA_sphericity,SASA_surface_area(Å²),SASA_volume(Å³),NBO_charge_C1,NBO_charge_C4,NBO_charge_H5,NBO_charge_O2,NBO_charge_O3,NMR_shift_C1,NMR_shift_C4,NMR_shift_H5,distance_O3_H5(Å),dihedral_C4_C1_O3_H5(°),dihedral_O2_C1_O3_H5(°),IR_freq_C1_O2,%Vbur_C1_2.0Å,%Vbur_C4_2.0Å,%Vbur_C1_2.5Å,%Vbur_C4_2.5Å,%Vbur_C1_3.0Å,%Vbur_C4_3.0Å,%Vbur_C1_3.5Å,%Vbur_C4_3.5Å,%Vbur_C1_4.0Å,%Vbur_C4_4.0Å,%Vbur_C1_4.5Å,%Vbur_C4_4.5Å,%Vbur_C1_5.0Å,%Vbur_C4_5.0Å,%Vbur_C1_5.5Å,%Vbur_C4_5.5Å,%Vbur_C1_6.0Å,%Vbur_C4_6.0Å,%Vbur_C1_max_hemisphere_3Å,%Vbur_C1_min_hemisphere_3Å,%Vbur_C1_max_hemisphere_3.5Å,%Vbur_C1_min_hemisphere_3.5Å,%Vbur_C1_max_hemisphere_4Å,%Vbur_C1_min_hemisphere_4Å,%Vbur_C1_max_hemisphere_4.5Å,%Vbur_C1_min_hemisphere_4.5Å,%Vbur_C1_max_hemisphere_5Å,%Vbur_C1_min_hemisphere_5Å,Sterimol_B1_C1_C4(Å)_morfeus,Sterimol_B5_C1_C4(Å)_morfeus,Sterimol_L_C1_C4(Å)_morfeus
0,Ac1_clust-1,3,2,1,13,12,-543.543441,-543.291813,-543.235970,298.15,0.055843,0.054099,0.292507,-543.290069,-0.35353,0.03938,0.39291,-0.157075,0.03140,27.5759,123.403,1.3872,1669.842,0.909273,370.318607,581.003359,0.80633,-0.52240,0.49898,-0.61057,-0.69506,-10.7201,147.9625,25.3277,0.97246,179.601,1.130,1832.1908,96.871772,97.691761,90.127295,84.967118,78.622244,71.062090,65.065924,60.080251,54.098520,49.841723,45.786026,41.055651,38.697140,33.852212,32.211203,28.161793,26.351308,23.564107,93.109356,64.250596,84.048873,46.058517,75.523711,32.634535,67.932116,23.649705,60.041074,17.350052,1.958411,6.173337,4.658647
1,Ac1_clust-10,3,2,1,13,12,-543.537866,-543.292382,-543.230502,298.15,0.061880,0.057455,0.291621,-543.287958,-0.35342,0.04394,0.39736,-0.154740,0.03013,31.6898,125.391,2.1768,1640.530,0.857824,412.637722,626.216529,0.79725,-0.51017,0.49393,-0.59157,-0.69617,-9.7259,148.1914,25.8632,0.97191,178.877,0.467,1849.8743,94.473140,97.536803,82.479815,85.004558,65.471357,71.097475,48.637835,59.546217,37.247455,48.395118,29.933625,38.736052,25.109386,31.195405,21.607482,25.577032,18.757880,21.434419,69.416716,61.522274,53.910035,43.323705,43.173918,31.251940,35.770884,24.041551,31.036549,19.147536,1.784935,8.457204,6.805627
2,Ac1_clust-11,3,2,1,13,12,-543.539014,-543.290972,-543.231499,298.15,0.059473,0.056151,0.292038,-543.287650,-0.35148,0.04371,0.39519,-0.153885,0.02996,22.4113,124.817,2.1546,1780.705,0.866179,403.727587,614.918993,0.79730,-0.51238,0.49382,-0.59184,-0.69512,-9.1149,145.9067,25.8703,0.97198,178.653,0.483,1849.5058,94.521565,98.276085,82.623063,87.275361,65.543057,75.093117,49.114214,64.315831,38.336773,53.410325,31.233447,43.897892,26.602362,36.122915,23.274070,30.050101,20.470089,25.162592,69.681168,61.512962,53.854128,44.327712,42.470983,34.138942,33.805683,28.634617,27.721453,25.466715,2.342731,6.382027,8.078495
3,Ac1_clust-12,3,2,1,13,12,-543.540547,-543.294717,-543.233299,298.15,0.061417,0.057290,0.291455,-543.290589,-0.35482,0.04101,0.39583,-0.156905,0.03110,23.9610,124.822,1.8418,1865.071,0.861717,411.488009,627.850574,0.79637,-0.50553,0.49375,-0.59472,-0.69599,-9.0786,146.5742,25.8367,0.97200,178.241,0.289,1846.8516,96.064695,97.136493,85.509181,82.992577,69.311494,67.682881,52.907193,54.992080,40.855627,43.393489,33.270558,34.027657,27.784129,27.544267,23.868226,23.163063,20.841762,19.833326,77.393102,61.246648,63.227963,42.535175,53.138384,28.493731,46.441364,20.029199,40.909700,14.590363,1.764190,7.623434,4.792890
4,Ac1_clust-13,3,2,1,13,12,-543.538204,-543.290149,-543.230894,298.15,0.059255,0.055999,0.291839,-543.286894,-0.35491,0.03949,0.39440,-0.157710,0.03153,20.2519,123.936,1.5242,1677.904,0.867586,403.118088,615.022654,0.79790,-0.52050,0.49317,-0.60063,-0.69009,-8.7979,152.1215,25.6678,0.97241,178.606,0.472,1842.8561,95.454545,97.520661,86.689348,85.657312,72.240018,72.403903,56.592434,60.887416,44.779576,49.495686,36.727434,39.665465,30.643349,31

## Save collected properties to Excel

Helpful to save here in case the Notebook crashes or if you want to add more properties before post-processsing. Can be read in at 5.1.1.

In [9]:
writer = pd.ExcelWriter('All_Conformer_Properties_for_Ac_example.xlsx')
df.to_excel(writer)
writer.save()

# Post-processing

## User input for data processing

In [10]:
#for numerically named compounds, prefix is any text common to all BEFORE the number and suffix is common to all AFTER the number
#this is a template for our files that are all named "AcXXX_clust-X.log" or "AcXXX_conf-X.log"
prefix = "Ac" 
suffix = "_"

#columns that provide atom mapping information are dropped
atom_columns_to_drop = ["C1", "O2", "O3", "C4", "H5"]

#title of the column for the energy you want to use for boltzmann averaging and lowest E conformer determination
energy_col_header = "G(T)_spc(Hartree)"

### Option to import an Excel sheet if you're using properties or energies collected outside of this notebook

If you would like to use post-processing functionality (i.e. Boltzmann averaging, lowest E conformers, etc.) you can read in a dataframe with properties (e.g. QikProp properties) or energies (e.g. if you don't/can't run linked jobs) collected outside of this notebook. 

Check out the dataframe_sample.xlsx to make sure you have the desired format. 

In [11]:
df = pd.read_excel('All_Conformer_Properties_for_Ac_example.xlsx','Sheet1',index_col=0,header=0,engine='openpyxl')
display(df.head())

,log_name,C4,C1,O3,H5,O2,E_spc (Hartree),G(T)_spc(Hartree),H_spc(Hartree),T,T*S,T*qh_S,ZPE(Hartree),qh_G(T)_spc(Hartree),HOMO,LUMO,η,μ,ω,polar_aniso(au),polar_iso(au),dipole(Debye),volume(Bohr_radius³/mol),SASA_sphericity,SASA_surface_area(Å²),SASA_volume(Å³),NBO_charge_C1,NBO_charge_C4,NBO_charge_H5,NBO_charge_O2,NBO_charge_O3,NMR_shift_C1,NMR_shift_C4,NMR_shift_H5,distance_O3_H5(Å),dihedral_C4_C1_O3_H5(°),dihedral_O2_C1_O3_H5(°),IR_freq_C1_O2,%Vbur_C1_2.0Å,%Vbur_C4_2.0Å,%Vbur_C1_2.5Å,%Vbur_C4_2.5Å,%Vbur_C1_3.0Å,%Vbur_C4_3.0Å,%Vbur_C1_3.5Å,%Vbur_C4_3.5Å,%Vbur_C1_4.0Å,%Vbur_C4_4.0Å,%Vbur_C1_4.5Å,%Vbur_C4_4.5Å,%Vbur_C1_5.0Å,%Vbur_C4_5.0Å,%Vbur_C1_5.5Å,%Vbur_C4_5.5Å,%Vbur_C1_6.0Å,%Vbur_C4_6.0Å,%Vbur_C1_max_hemisphere_3Å,%Vbur_C1_min_hemisphere_3Å,%Vbur_C1_max_hemisphere_3.5Å,%Vbur_C1_min_hemisphere_3.5Å,%Vbur_C1_max_hemisphere_4Å,%Vbur_C1_min_hemisphere_4Å,%Vbur_C1_max_hemisphere_4.5Å,%Vbur_C1_min_hemisphere_4.5Å,%Vbur_C1_max_hemisphere_5Å,%Vbur_C1_min_hemisphere_5Å,Sterimol_B1_C1_C4(Å)_morfeus,Sterimol_B5_C1_C4(Å)_morfeus,Sterimol_L_C1_C4(Å)_morfeus
0,Ac1_clust-1,3,2,1,13,12,-543.543441,-543.291813,-543.235970,298.15,0.055843,0.054099,0.292507,-543.290069,-0.35353,0.03938,0.39291,-0.157075,0.03140,27.5759,123.403,1.3872,1669.842,0.909273,370.318607,581.003359,0.80633,-0.52240,0.49898,-0.61057,-0.69506,-10.7201,147.9625,25.3277,0.97246,179.601,1.130,1832.1908,96.871772,97.691761,90.127295,84.967118,78.622244,71.062090,65.065924,60.080251,54.098520,49.841723,45.786026,41.055651,38.697140,33.852212,32.211203,28.161793,26.351308,23.564107,93.109356,64.250596,84.048873,46.058517,75.523711,32.634535,67.932116,23.649705,60.041074,17.350052,1.958411,6.173337,4.658647
1,Ac1_clust-10,3,2,1,13,12,-543.537866,-543.292382,-543.230502,298.15,0.061880,0.057455,0.291621,-543.287958,-0.35342,0.04394,0.39736,-0.154740,0.03013,31.6898,125.391,2.1768,1640.530,0.857824,412.637722,626.216529,0.79725,-0.51017,0.49393,-0.59157,-0.69617,-9.7259,148.1914,25.8632,0.97191,178.877,0.467,1849.8743,94.473140,97.536803,82.479815,85.004558,65.471357,71.097475,48.637835,59.546217,37.247455,48.395118,29.933625,38.736052,25.109386,31.195405,21.607482,25.577032,18.757880,21.434419,69.416716,61.522274,53.910035,43.323705,43.173918,31.251940,35.770884,24.041551,31.036549,19.147536,1.784935,8.457204,6.805627
2,Ac1_clust-11,3,2,1,13,12,-543.539014,-543.290972,-543.231499,298.15,0.059473,0.056151,0.292038,-543.287650,-0.35148,0.04371,0.39519,-0.153885,0.02996,22.4113,124.817,2.1546,1780.705,0.866179,403.727587,614.918993,0.79730,-0.51238,0.49382,-0.59184,-0.69512,-9.1149,145.9067,25.8703,0.97198,178.653,0.483,1849.5058,94.521565,98.276085,82.623063,87.275361,65.543057,75.093117,49.114214,64.315831,38.336773,53.410325,31.233447,43.897892,26.602362,36.122915,23.274070,30.050101,20.470089,25.162592,69.681168,61.512962,53.854128,44.327712,42.470983,34.138942,33.805683,28.634617,27.721453,25.466715,2.342731,6.382027,8.078495
3,Ac1_clust-12,3,2,1,13,12,-543.540547,-543.294717,-543.233299,298.15,0.061417,0.057290,0.291455,-543.290589,-0.35482,0.04101,0.39583,-0.156905,0.03110,23.9610,124.822,1.8418,1865.071,0.861717,411.488009,627.850574,0.79637,-0.50553,0.49375,-0.59472,-0.69599,-9.0786,146.5742,25.8367,0.97200,178.241,0.289,1846.8516,96.064695,97.136493,85.509181,82.992577,69.311494,67.682881,52.907193,54.992080,40.855627,43.393489,33.270558,34.027657,27.784129,27.544267,23.868226,23.163063,20.841762,19.833326,77.393102,61.246648,63.227963,42.535175,53.138384,28.493731,46.441364,20.029199,40.909700,14.590363,1.764190,7.623434,4.792890
4,Ac1_clust-13,3,2,1,13,12,-543.538204,-543.290149,-543.230894,298.15,0.059255,0.055999,0.291839,-543.286894,-0.35491,0.03949,0.39440,-0.157710,0.03153,20.2519,123.936,1.5242,1677.904,0.867586,403.118088,615.022654,0.79790,-0.52050,0.49317,-0.60063,-0.69009,-8.7979,152.1215,25.6678,0.97241,178.606,0.472,1842.8561,95.454545,97.520661,86.689348,85.657312,72.240018,72.403903,56.592434,60.887416,44.779576,49.495686,36.727434,39.665465,30.643349,31

## Generating a list of compounds that have conformational ensembles

In [12]:
#this is a template for our files that are all named "AcXXX_clust-X.log"

compound_list = []
    
for index, row in df.iterrows():
    log_file = row['log_name'] #read file name from df
    prefix_and_compound = log_file.split(str(suffix)) #splits to get "AcXXX" (entry O) (and we don't use the "clust-X" (entry 1))
    compound = prefix_and_compound[0].split(str(prefix)) #splits again to get "XXX" (entry 1) (and we don't use the empty string "" (entry 0))
    compound_list.append(compound[1])

compound_list = list(set(compound_list)) #removes duplicate stuctures that result from having conformers of each
compound_list.sort() #reorders numerically (not sure if it reorders alphabetically)
print(compound_list)

#this should generate a list that looks like this: ['24', '27', '34', '48']

['1', '2', '3', '4', '5']


## Post-processing to get properties for each compound

In [13]:
all_df_master = pd.DataFrame(columns=[])
properties_df_master = pd.DataFrame(columns=[])

for compound in compound_list: 
    #defines the common start to all files using the input above 
    substring = str(prefix) + str(compound) + str(suffix)
    
    #makes a data frame for one compound at a time for post-processing
    valuesdf = df[df["log_name"].str.startswith(substring)]
    valuesdf = valuesdf.drop(columns = atom_columns_to_drop)
    valuesdf = valuesdf.reset_index(drop = True)  #you must re-index otherwise the 2nd, 3rd, etc. compounds fail
   
    #define columns that won't be included in summary properties or are treated differently because they don't make sense to Boltzmann average
    non_boltz_columns = ["G(Hartree)","∆G(Hartree)","∆G(kcal/mol)", "e^(-∆G/RT)","Mole Fraction"] #don't boltzman average columns containing these strings in the column label
    reg_avg_columns = ['CPU_time_total(hours)', 'Wall_time_total(hours)'] #don't boltzmann average these either, we average them in case that is helpful
    gv_extra_columns = ['E_spc (Hartree)', 'H_spc(Hartree)', 'T', 'T*S', 'T*qh_S', 'ZPE(Hartree)', 'qh_G(T)_spc(Hartree)', "G(T)_spc(Hartree)"]
    gv_extra_columns.remove(str(energy_col_header))
    
    #calculate the summary properties based on all conformers (Boltzmann Average, Minimum, Maximum, Boltzmann Weighted Std)
    valuesdf["∆G(Hartree)"] = valuesdf[energy_col_header] - valuesdf[energy_col_header].min()
    valuesdf["∆G(kcal/mol)"] = valuesdf["∆G(Hartree)"] * 627.5
    valuesdf["e^(-∆G/RT)"] = np.exp((valuesdf["∆G(kcal/mol)"] * -1000) / (1.987204 * 298.15)) #R is in cal/(K*mol)
    valuesdf["Mole Fraction"] = valuesdf["e^(-∆G/RT)"] / valuesdf["e^(-∆G/RT)"].sum()
    values_boltz_row = []
    values_min_row = []
    values_max_row = []
    values_boltz_stdev_row =[]
    #values_range_row = []
    values_exclude_columns = []
    
    for column in valuesdf:
        if "log_name" in column:
            values_boltz_row.append("Boltzmann Averages")
            values_min_row.append("Ensemble Minimum")
            values_max_row.append("Ensemble Maximum")
            values_boltz_stdev_row.append("Boltzmann Standard Deviation")
            #values_range_row.append("Ensemble Range")
            values_exclude_columns.append(column) #used later to build final dataframe
        elif any(phrase in column for phrase in non_boltz_columns) or any(phrase in column for phrase in gv_extra_columns):
            values_boltz_row.append("")
            values_min_row.append("")
            values_max_row.append("")
            values_boltz_stdev_row.append("")
            #values_range_row.append("")
        elif any(phrase in column for phrase in reg_avg_columns):
            values_boltz_row.append(valuesdf[column].mean()) #intended to print the average CPU/wall time in the boltz column
            values_min_row.append("")
            values_max_row.append("")
            values_boltz_stdev_row.append("")
            #values_range_row.append("")
        else:
            valuesdf[column] = pd.to_numeric(valuesdf[column]) #to hopefully solve the error that sometimes occurs where the float(Mole Fraction) cannot be mulitplied by the string(property)
            values_boltz_row.append((valuesdf[column] * valuesdf["Mole Fraction"]).sum())
            values_min_row.append(valuesdf[column].min())
            values_max_row.append(valuesdf[column].max())
            #values_range_row.append(valuesdf[column].max() - valuesdf[column].min())

            
            # this section generates the weighted std deviation (weighted by mole fraction) 
            # formula: https://www.statology.org/weighted-standard-deviation-excel/
    
            boltz = (valuesdf[column] * valuesdf["Mole Fraction"]).sum() #number
            delta_values_sq = []
    
            #makes a list of the "deviation" for each conformer           
            for index, row in valuesdf.iterrows(): 
                value = row[column]
                delta_value_sq = (value - boltz)**2
                delta_values_sq.append(delta_value_sq)
            
            #w is list of weights (i.e. mole fractions)
            w = list(valuesdf["Mole Fraction"])
            wstdev = np.sqrt( (np.average(delta_values_sq, weights=w)) / (((len(w)-1)/len(w))*np.sum(w)) )
            #np.average(delta_values_sq, weights=w) generates sum of each (delta_value_sq * mole fraction)
           
            #if there is only one conformer in the ensemble, set the weighted standard deviation to 0 
            if len(w) == 1:
                wstdev = 0
            values_boltz_stdev_row.append(wstdev)
            
            
    valuesdf.loc[len(valuesdf)] = values_boltz_row
    valuesdf.loc[len(valuesdf)] = values_boltz_stdev_row
    valuesdf.loc[len(valuesdf)] = values_min_row
    valuesdf.loc[len(valuesdf)] = values_max_row
    #valuesdf.loc[len(valuesdf)] = values_range_row

    #final output format is built here:
    explicit_order_front_columns = ["log_name", energy_col_header,"∆G(Hartree)","∆G(kcal/mol)","e^(-∆G/RT)","Mole Fraction"]
    
    #reorders the dataframe using front columns defined above
    valuesdf = valuesdf[explicit_order_front_columns + [col for col in valuesdf.columns if col not in explicit_order_front_columns and col not in values_exclude_columns]]
    
    #determine the index of the lowest energy conformer
    low_e_index = valuesdf[valuesdf["∆G(Hartree)"] == 0].index.tolist()
    
    #copy the row to a new_row with the name of the log changed to Lowest E Conformer
    new_row = valuesdf.loc[low_e_index[0]]
    new_row['log_name'] = "Lowest E Conformer"   
    valuesdf =  valuesdf.append(new_row, ignore_index=True)

#------------------------------EDIT THIS SECTION IF YOU WANT A SPECIFIC CONFORMER----------------------------------  
    #if you want all properties for a conformer with a particular property (i.e. all properties for the Vbur_min conformer)
    #this template can be adjusted for min/max/etc. 
    
    #find the index for the min or max column:
    #ensemble_min_index = valuesdf[valuesdf["log_name"] == "Ensemble Minimum"].index.tolist()
    
    #find the min or max value of the property (based on index above)
    #saves the value in a list (min_value) with one entry (this is why we call min_value[0])
    #min_value = valuesdf.loc[ensemble_min_index, "%Vbur_C4_3.0Å"].tolist()   
    #vbur_min_index = valuesdf[valuesdf["%Vbur_C4_3.0Å"] == min_value[0]].index.tolist()
    
    #copy the row to a new_row with the name of the log changed to Property_min_conformer
    #new_row = valuesdf.loc[vbur_min_index[0]]
    #new_row['log_name'] = "%Vbur_C4_3.0Å_min_Conformer"   
    #valuesdf =  valuesdf.append(new_row, ignore_index=True)
#--------------------------------------------------------------------------------------------------------------------    
    
    #appends the frame to the master output
    all_df_master = pd.concat([all_df_master, valuesdf])
    
    #drop all the individual conformers
    dropindex = valuesdf[valuesdf["log_name"].str.startswith(substring)].index
    valuesdf = valuesdf.drop(dropindex)
    valuesdf = valuesdf.reset_index(drop = True)
    
    #display(valuesdf)   
    
    #drop the columns created to determine the mole fraction and some that 
    valuesdf = valuesdf.drop(columns = explicit_order_front_columns)
    try:
        valuesdf = valuesdf.drop(columns = gv_extra_columns)
    except:
        pass
    try:
        valuesdf = valuesdf.drop(columns = reg_avg_columns)
    except:
        pass
        
#---------------------THIS MAY NEED TO CHANGE DEPENDING ON HOW YOU LABEL YOUR COMPOUNDS------------------------------  
    compound_name = prefix + str(compound) 
#--------------------------------------------------------------------------------------------------------------------      

    properties_df = pd.DataFrame({'Compound_Name': [compound_name]})
    
    #builds a dataframe (for each compound) by adding summary properties as new columns
    for (columnName, columnData) in valuesdf.iteritems():
        #the indexes need to match the values dataframe - display it to double check if you need to make changes 
        #(uncomment the display(valuesdf) in row 124 of this cell)
        properties_df[str(columnName) + "_Boltz"] = [columnData.values[0]]
        properties_df[str(columnName) + "_Boltz_stdev"] = [columnData.values[1]]
        properties_df[str(columnName) + "_min"] = [columnData.values[2]]
        properties_df[str(columnName) + "_max"] = [columnData.values[3]]
        #properties_df[str(columnName) + "_range"] = [columnData.values[4]]
        properties_df[str(columnName) + "_low_E"] = [columnData.values[4]]
        
        #if you're collecting properties for a specific conformer, add these here (note the index)
        #example:
        #properties_df[str(columnName) + "_V_bur_min"] = [columnData.values[6]]
        
        #if you only want a table with Boltz, you can comment out the other summary properties to generate a Boltz spreadsheet
        #of if you don't want to collect range, etc.
    #concatenates the individual acid properties df into the master properties df
    properties_df_master = pd.concat([properties_df_master, properties_df], axis = 0)

all_df_master = all_df_master.reset_index(drop = True)
properties_df_master = properties_df_master.reset_index(drop = True)


### Peek at your new dataframes

In [14]:
display(properties_df_master.head())
display(all_df_master)

,Compound_Name,HOMO_Boltz,HOMO_Boltz_stdev,HOMO_min,HOMO_max,HOMO_low_E,LUMO_Boltz,LUMO_Boltz_stdev,LUMO_min,LUMO_max,LUMO_low_E,η_Boltz,η_Boltz_stdev,η_min,η_max,η_low_E,μ_Boltz,μ_Boltz_stdev,μ_min,μ_max,μ_low_E,ω_Boltz,ω_Boltz_stdev,ω_min,ω_max,ω_low_E,polar_aniso(au)_Boltz,polar_aniso(au)_Boltz_stdev,polar_aniso(au)_min,polar_aniso(au)_max,polar_aniso(au)_low_E,polar_iso(au)_Boltz,polar_iso(au)_Boltz_stdev,polar_iso(au)_min,polar_iso(au)_max,polar_iso(au)_low_E,dipole(Debye)_Boltz,dipole(Debye)_Boltz_stdev,dipole(Debye)_min,dipole(Debye)_max,dipole(Debye)_low_E,volume(Bohr_radius³/mol)_Boltz,volume(Bohr_radius³/mol)_Boltz_stdev,volume(Bohr_radius³/mol)_min,volume(Bohr_radius³/mol)_max,volume(Bohr_radius³/mol)_low_E,SASA_sphericity_Boltz,SASA_sphericity_Boltz_stdev,SASA_sphericity_min,SASA_sphericity_max,SASA_sphericity_low_E,SASA_surface_area(Å²)_Boltz,SASA_surface_area(Å²)_Boltz_stdev,SASA_surface_area(Å²)_min,SASA_surface_area(Å²)_max,SASA_surface_area(Å²)_low_E,SASA_volume(Å³)_Boltz,SASA_volume(Å³)_Boltz_stdev,SASA_volume(Å³)_min,SASA_volume(Å³)_max,SASA_volume(Å³)_low_E,NBO_charge_C1_Boltz,NBO_charge_C1_Boltz_stdev,NBO_charge_C1_min,NBO_charge_C1_max,NBO_charge_C1_low_E,NBO_charge_C4_Boltz,NBO_charge_C4_Boltz_stdev,NBO_charge_C4_min,NBO_charge_C4_max,NBO_charge_C4_low_E,NBO_charge_H5_Boltz,NBO_charge_H5_Boltz_stdev,NBO_charge_H5_min,NBO_charge_H5_max,NBO_charge_H5_low_E,NBO_charge_O2_Boltz,NBO_charge_O2_Boltz_stdev,NBO_charge_O2_min,NBO_charge_O2_max,NBO_charge_O2_low_E,NBO_charge_O3_Boltz,NBO_charge_O3_Boltz_stdev,NBO_charge_O3_min,NBO_charge_O3_max,NBO_charge_O3_low_E,NMR_shift_C1_Boltz,NMR_shift_C1_Boltz_stdev,NMR_shift_C1_min,NMR_shift_C1_max,NMR_shift_C1_low_E,NMR_shift_C4_Boltz,NMR_shift_C4_Boltz_stdev,NMR_shift_C4_min,NMR_shift_C4_max,NMR_shift_C4_low_E,NMR_shift_H5_Boltz,NMR_shift_H5_Boltz_stdev,NMR_shift_H5_min,NMR_shift_H5_max,NMR_shift_H5_low_E,distance_O3_H5(Å)_Boltz,distance_O3_H5(Å)_Boltz_stdev,distance_O3_H5(Å)_min,distance_O3_H5(Å)_max,distance_O3_H5(Å)_low_E,dihedral_C4_C1_O3_H5(°)_Boltz,dihedral_C4_C1_O3_H5(°)_Boltz_stdev,dihedral_C4_C1_O3_H5(°)_min,dihedral_C4_C1_O3_H5(°)_max,dihedral_C4_C1_O3_H5(°)_low_E,dihedral_O2_C1_O3_H5(°)_Boltz,dihedral_O2_C1_O3_H5(°)_Boltz_stdev,dihedral_O2_C1_O3_H5(°)_min,dihedral_O2_C1_O3_H5(°)_max,dihedral_O2_C1_O3_H5(°)_low_E,IR_freq_C1_O2_Boltz,IR_freq_C1_O2_Boltz_stdev,IR_freq_C1_O2_min,IR_freq_C1_O2_max,IR_freq_C1_O2_low_E,%Vbur_C1_2.0Å_Boltz,%Vbur_C1_2.0Å_Boltz_stdev,%Vbur_C1_2.0Å_min,%Vbur_C1_2.0Å_max,%Vbur_C1_2.0Å_low_E,%Vbur_C4_2.0Å_Boltz,%Vbur_C4_2.0Å_Boltz_stdev,%Vbur_C4_2.0Å_min,%Vbur_C4_2.0Å_max,%Vbur_C4_2.0Å_low_E,%Vbur_C1_2.5Å_Boltz,%Vbur_C1_2.5Å_Boltz_stdev,%Vbur_C1_2.5Å_min,%Vbur_C1_2.5Å_max,%Vbur_C1_2.5Å_low_E,%Vbur_C4_2.5Å_Boltz,%Vbur_C4_2.5Å_Boltz_stdev,%Vbur_C4_2.5Å_min,%Vbur_C4_2.5Å_max,%Vbur_C4_2.5Å_low_E,%Vbur_C1_3.0Å_Boltz,%Vbur_C1_3.0Å_Boltz_stdev,%Vbur_C1_3.0Å_min,%Vbur_C1_3.0Å_max,%Vbur_C1_3.0Å_low_E,%Vbur_C4_3.0Å_Boltz,%Vbur_C4_3.0Å_Boltz_stdev,%Vbur_C4_3.0Å_min,%Vbur_C4_3.0Å_max,%Vbur_C4_3.0Å_low_E,%Vbur_C1_3.5Å_Boltz,%Vbur_C1_3.5Å_Boltz_stdev,%Vbur_C1_3.5Å_min,%Vbur_C1_3.5Å_max,%Vbur_C1_3.5Å_low_E,%Vbur_C4_3.5Å_Boltz,%Vbur_C4_3.5Å_Boltz_stdev,%Vbur_C4_3.5Å_min,%Vbur_C4_3.5Å_max,%Vbur_C4_3.5Å_low_E,%Vbur_C1_4.0Å_Boltz,%Vbur_C1_4.0Å_Boltz_stdev,%Vbur_C1_4.0Å_min,%Vbur_C1_4.0Å_max,%Vbur_C1_4.0Å_low_E,%Vbur_C4_4.0Å_Boltz,%Vbur_C4_4.0Å_Boltz_stdev,%Vbur_C4_4.0Å_min,%Vbur_C4_4.0Å_max,%Vbur_C4_4.0Å_low_E,%Vbur_C1_4.5Å_Boltz,%Vbur_C1_4.5Å_Boltz_stdev,%Vbur_C1_4.5Å_min,%Vbur_C1_4.5Å_max,%Vbur_C1_4.5Å_low_E,%Vbur_C4_4.5Å_Boltz,%Vbur_C4_4.5Å_Boltz_stdev,%Vbur_C4_4.5Å_min,%Vbur_C4_4.5Å_max,%Vbur_C4_4.5Å_low_E,%Vbur_C1_5.0Å_Boltz,%Vbur_C1_5.0Å_Boltz_stdev,%Vbur_C1_5.0Å_min,%Vbur_C1_5.0Å_max,%Vbur_C1_5.0Å_low_E,%Vbur_C4_5.0Å_Boltz,%Vbur_C4_5.0Å_Boltz_stdev,%Vbur_C4_5.0Å_min,%Vbur_C4_5.0Å_max,%Vbur_C4_5.0Å_low_E,%Vbur_C1_5.5Å_Boltz,%Vbur_C1_5.5Å_Boltz_stdev,%Vbur_C1_5.5Å_min,%Vbur_C1_5.5Å_max,%Vbur_C1_5.5Å_low_E,%Vbur_C4_5.5Å_Boltz,%Vbur_C4_5.5Å_Boltz_stdev,%

,log_name,G(T)_spc(Hartree),∆G(Hartree),∆G(kcal/mol),e^(-∆G/RT),Mole Fraction,E_spc (Hartree),H_spc(Hartree),T,T*S,T*qh_S,ZPE(Hartree),qh_G(T)_spc(Hartree),HOMO,LUMO,η,μ,ω,polar_aniso(au),polar_iso(au),dipole(Debye),volume(Bohr_radius³/mol),SASA_sphericity,SASA_surface_area(Å²),SASA_volume(Å³),NBO_charge_C1,NBO_charge_C4,NBO_charge_H5,NBO_charge_O2,NBO_charge_O3,NMR_shift_C1,NMR_shift_C4,NMR_shift_H5,distance_O3_H5(Å),dihedral_C4_C1_O3_H5(°),dihedral_O2_C1_O3_H5(°),IR_freq_C1_O2,%Vbur_C1_2.0Å,%Vbur_C4_2.0Å,%Vbur_C1_2.5Å,%Vbur_C4_2.5Å,%Vbur_C1_3.0Å,%Vbur_C4_3.0Å,%Vbur_C1_3.5Å,%Vbur_C4_3.5Å,%Vbur_C1_4.0Å,%Vbur_C4_4.0Å,%Vbur_C1_4.5Å,%Vbur_C4_4.5Å,%Vbur_C1_5.0Å,%Vbur_C4_5.0Å,%Vbur_C1_5.5Å,%Vbur_C4_5.5Å,%Vbur_C1_6.0Å,%Vbur_C4_6.0Å,%Vbur_C1_max_hemisphere_3Å,%Vbur_C1_min_hemisphere_3Å,%Vbur_C1_max_hemisphere_3.5Å,%Vbur_C1_min_hemisphere_3.5Å,%Vbur_C1_max_hemisphere_4Å,%Vbur_C1_min_hemisphere_4Å,%Vbur_C1_max_hemisphere_4.5Å,%Vbur_C1_min_hemisphere_4.5Å,%Vbur_C1_max_hemisphere_5Å,%Vbur_C1_min_hemisphere_5Å,Sterimol_B1_C1_C4(Å)_morfeus,Sterimol_B5_C1_C4(Å)_morfeus,Sterimol_L_C1_C4(Å)_morfeus
0,Ac1_clust-1,-543.291813,0.007211,4.52503,0.000482,0.00015,-543.543441,-543.23597,298.15,0.055843,0.054099,0.292507,-543.290069,-0.353530,0.039380,0.392910,-0.157075,0.031400,27.575900,123.403000,1.387200,1669.842000,0.909273,370.318607,581.003359,0.806330,-0.522400,0.498980,-0.610570,-0.695060,-10.720100,147.962500,25.327700,0.972460,179.601000,1.130000,1832.190800,96.871772,9.769176e+01,90.127295,84.967118,78.622244,71.062090,65.065924,60.080251,54.098520,49.841723,45.786026,41.055651,38.697140,33.852212,32.211203,28.161793,26.351308,23.564107,93.109356,64.250596,84.048873,46.058517,75.523711,32.634535,67.932116,23.649705,60.041074,17.350052,1.958411,6.173337,4.658647
1,Ac1_clust-10,-543.292382,0.006642,4.167876,0.000881,0.000275,-543.537866,-543.230502,298.15,0.06188,0.057455,0.291621,-543.287958,-0.353420,0.043940,0.397360,-0.154740,0.030130,31.689800,125.391000,2.176800,1640.530000,0.857824,412.637722,626.216529,0.797250,-0.510170,0.493930,-0.591570,-0.696170,-9.725900,148.191400,25.863200,0.971910,178.877000,0.467000,1849.874300,94.473140,9.753680e+01,82.479815,85.004558,65.471357,71.097475,48.637835,59.546217,37.247455,48.395118,29.933625,38.736052,25.109386,31.195405,21.607482,25.577032,18.757880,21.434419,69.416716,61.522274,53.910035,43.323705,43.173918,31.251940,35.770884,24.041551,31.036549,19.147536,1.784935,8.457204,6.805627
2,Ac1_clust-11,-543.290972,0.008052,5.052621,0.000198,0.000062,-543.539014,-543.231499,298.15,0.059473,0.056151,0.292038,-543.28765,-0.351480,0.043710,0.395190,-0.153885,0.029960,22.411300,124.817000,2.154600,1780.705000,0.866179,403.727587,614.918993,0.797300,-0.512380,0.493820,-0.591840,-0.695120,-9.114900,145.906700,25.870300,0.971980,178.653000,0.483000,1849.505800,94.521565,9.827608e+01,82.623063,87.275361,65.543057,75.093117,49.114214,64.315831,38.336773,53.410325,31.233447,43.897892,26.602362,36.122915,23.274070,30.050101,20.470089,25.162592,69.681168,61.512962,53.854128,44.327712,42.470983,34.138942,33.805683,28.634617,27.721453,25.466715,2.342731,6.382027,8.078495
3,Ac1_clust-12,-543.294717,0.004308,2.703041,0.010439,0.003258,-543.540547,-543.233299,298.15,0.061417,0.05729,0.291455,-543.290589,-0.354820,0.041010,0.395830,-0.156905,0.031100,23.961000,124.822000,1.841800,1865.071000,0.861717,411.488009,627.850574,0.796370,-0.505530,0.493750,-0.594720,-0.695990,-9.078600,146.574200,25.836700,0.972000,178.241000,0.289000,1846.851600,96.064695,9.713649e+01,85.509181,82.992577,69.311494,67.682881,52.907193,54.992080,40.855627,43.393489,33.270558,34.027657,27.784129,27.544267,23.868226,23.163063,20.841762,19.833326,77.393102,61.246648,63.227963,42.535175,53.138384,28.493731,46.441364,20.029199,40.909700,14.590363,1.764190,7.623434,4.792890
4,Ac1_clust-13,-543.290149,0.008875,5.568985,0.000083,0.000026,-543.538204,-543.230894,298.15,0.059255,0.055999,0.291839,-543.286894,-0.354910,0.039490,0.394400,-0.157710,0.0

### Save to Microsoft Excelᵀᴹ 

In [15]:
all_df_master.to_excel('All_Conformer_and_Summary_Properties_for_Ac_example.xlsx', index = False)
properties_df_master.to_excel('Summary_Properties_for_Ac_example.xlsx', index = False)